
Questions to be considered for the project 

what industriess performed best over a certain time period?  
Are there trends in stock price movements by quarter, year, decade?  
Can we predict stock price trends using historical data?  
Impact of certain events on the stock prices i.e. US election, end of quarter performance announcements by companies, press releases etc

In [ ]:
# installing yfinace package
#pip install yfinance

In [1]:
# importing libraries

import pandas as pd
import yfinance as yf
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt

In [9]:
# creating a list to define the industries and the companies that will be used for the project
industries = {
    'Technology': ['AAPL', 'MSFT', 'NVDA'],  
    'Quantum Computing': ['IONQ', 'RGTI', 'QBTS'],  
    'Electric Vehicles': ['TSLA', 'RIVN', 'LCID'],  
    'Renewable Energy': ['NEE', 'FSLR', 'ENPH']  
}

# creating an empty list to store summary statistics for each industry
industry_summary = []

# fetching the data for each industry by looping through the key value pairs returned from the .items() method
for industry, companies in industries.items():
    data = yf.Tickers(' '.join(companies))  # .Tickers() function is used to fetch the data for all of the companies in the list within the industry for that iteration of the for loop
    # creating lists to store the  market capitalisations, P/E ratios and dividend yields per company
    market_caps = []
    pe_ratios = []
    dividend_yields = []

    # getting the data for each company within the industry specified for the parent for loop
    for company in companies:
        info = data.tickers[company].info  # retrieveing detailed information about the company

        # Adding the market cap, P/E ratio and dividend yield for the current company to the lists defined above
        # getting the market cap and trailing P/E ratio, defaulting to 0 if data is missing
        market_caps.append(info.get('marketCap', 0))  
        pe_ratios.append(info.get('trailingPE', 0))
        #calculating the dividend yield as a percentage
        dividend_yields.append(info.get('dividendYield', 0) * 100 if info.get('dividendYield') else 0)

    # calculating the averages for the market cap, P/E ratio and dividend yield for the industry using the values added to the 3 lists for each company
    # first the 3 values are added together then the length of list is used to divide the sum giving the average for each industry
    avg_market_cap = sum(market_caps) / len(market_caps)
    avg_pe_ratio = sum(pe_ratios) / len(pe_ratios)
    avg_dividend_yield = sum(dividend_yields) / len(dividend_yields)

    # adding the results for the industry to the summary list
    industry_summary.append({
        'Industry': industry,  # industry name
        'Avg Market Cap ($B)': round(avg_market_cap / 1e9, 2),  # average market cap in billions
        'Avg P/E Ratio': round(avg_pe_ratio, 2),  # average price-to-earnings ratio
        'Avg Dividend Yield (%)': round(avg_dividend_yield, 2)  # Average dividend yield percentage
    })

# creating a DataFrame to store and display results 
summary_df = pd.DataFrame(industry_summary)  # converting summary list into a DataFrame
print(summary_df)  # displaying the summary table


            Industry  Avg Market Cap ($B)  Avg P/E Ratio  \
0         Technology              3533.16          44.69   
1  Quantum Computing                 5.81           0.00   
2  Electric Vehicles               494.02          41.70   
3   Renewable Energy                59.45          65.34   

   Avg Dividend Yield (%)  
0                    0.39  
1                    0.00  
2                    0.00  
3                    0.96  
